<a href="https://colab.research.google.com/github/osram90/whisper/blob/main/Copia_de_AnimateDiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### **PROYECTO DILATADO ANIMACIÓN**
Basado en:
> "[AnimateDiff](https://animatediff.github.io/): Animate Your Personalized Text-to-Image Diffusion Models without Specific Tuning"  
> by [Yuwei Guo](guoyuwei@pjlab.org.cn), [Ceyuan Yang](https://ceyuan.me/) [Anyi Rao](https://anyirao.com/), [Yaohui Wang](https://wyhsirius.github.io/) [Yu Qiao](https://wyhsirius.github.io/) [Dahua Lin](http://dahua.site/) [Bo Dai](https://daibo.info/)


Corre el siguiente paquete para intalar: da clic en play

In [ ]:
%%capture
# @title Instalación de AnimateDiff

#################
# Install stuff #
#################

!pip install napm einops omegaconf safetensors diffusers[torch]==0.11.1 transformers
!pip install xformers

# TODO: use huggingface hub rust downloader, backout aria dependency
!apt -y install -qq aria2


# Handle uninstallable research dependencies
from pathlib import Path
import os
#os.environ['NAPM_PATH'] = str(Path('.').absolute()) # "install" repo into cwd

import napm
napm.pseudoinstall_git_repo("https://github.com/guoyww/animatediff/", package_name='animatediff', add_install_dir_to_path=True)


In [ ]:
# @title Select Models

#################
# Select models #
#################

fpath_sd_model = "/content/models/StableDiffusion/" # @param {type:"string"}
fpath_dreambooth_lora = "/content/models/DreamBooth_LoRA/" # @param {type:"string"}

base_model = "Stable Diffusion v1.5" # @param ["Stable Diffusion v1.5", "Stable Diffusion v1.4"]

fpath_motion_prior = "/content/models/Motion_Module/"



################
# import stuff #
################

import datetime
import os
from pathlib import Path
from types import SimpleNamespace

import diffusers
from diffusers import AutoencoderKL, DDIMScheduler
from omegaconf import OmegaConf
from safetensors import safe_open
import torch
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer

# napm needs to be imported before animatediff
import napm
from animatediff.models.unet import UNet3DConditionModel
from animatediff.pipelines.pipeline_animation import AnimationPipeline
from animatediff.utils.util import save_videos_grid
from animatediff.utils.convert_from_ckpt import convert_ldm_unet_checkpoint, convert_ldm_clip_checkpoint, convert_ldm_vae_checkpoint
from animatediff.utils.convert_lora_safetensor_to_diffusers import convert_lora


# Being consistent with above in case user doesn't re-run the install cell
#os.environ['NAPM_PATH'] = str(Path('.').absolute()) # "install" repo into cwd


# dig up the config yaml from the napm dependency
cfg = napm.config.NapmConfig().load()
PKG_ROOT = Path(cfg['packages']['animatediff']['install_dir'])
inference_config = OmegaConf.load(PKG_ROOT/"configs/inference/inference-v1.yaml")


######################
# Download as needed #
######################


if not Path(fpath_sd_model).exists():
    !mkdir -p {fpath_sd_model}
    !git clone -b fp16 https://huggingface.co/runwayml/stable-diffusion-v1-5 {fpath_sd_model}

if not Path(fpath_motion_prior).exists():
    !mkdir -p {fpath_motion_prior}
    for mm_fname in ("mm_sd_v14.ckpt", "mm_sd_v15.ckpt"):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/{mm_fname} -d {fpath_motion_prior} -o {mm_fname}


if not Path(fpath_dreambooth_lora).exists():
    !mkdir -p {fpath_dreambooth_lora}
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/toonyou_beta3.safetensors -d {fpath_dreambooth_lora} -o toonyou_beta3.safetensors

############################################

mm_fname = "mm_sd_v15.ckpt"
if base_model != "Stable Diffusion v1.5":
    mm_fname = "mm_sd_v14.ckpt"


In [ ]:
%%capture

# @title Load Models

lora_dreambooth_model_path = str(list(Path(fpath_dreambooth_lora).glob('*.safetensors'))[0])
motion_module_path = str(Path(fpath_motion_prior)/mm_fname)
#unet_additional_kwargs = {}

tokenizer    = CLIPTokenizer.from_pretrained(fpath_sd_model, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(fpath_sd_model, subfolder="text_encoder")
vae          = AutoencoderKL.from_pretrained(fpath_sd_model, subfolder="vae")
unet         = UNet3DConditionModel.from_pretrained_2d(fpath_sd_model, subfolder="unet", unet_additional_kwargs=OmegaConf.to_container(inference_config.get("unet_additional_kwargs", {})))

pipeline = AnimationPipeline(
    vae=vae,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    unet=unet,
    scheduler=DDIMScheduler(**OmegaConf.to_container(inference_config.noise_scheduler_kwargs)),
    ).to("cuda")


# probably wanna change this
func_args = SimpleNamespace

motion_module_state_dict = torch.load(motion_module_path, map_location="cpu")
if "global_step" in motion_module_state_dict:
  func_args.update({"global_step": motion_module_state_dict["global_step"]})
missing, unexpected = pipeline.unet.load_state_dict(motion_module_state_dict, strict=False)
assert len(unexpected) == 0


if lora_dreambooth_model_path != "":
    if lora_dreambooth_model_path.endswith(".ckpt"):
        state_dict = torch.load(lora_dreambooth_model_path)
        pipeline.unet.load_state_dict(state_dict)

    elif lora_dreambooth_model_path.endswith(".safetensors"):
        state_dict = {}
        with safe_open(lora_dreambooth_model_path, framework="pt", device="cpu") as f:
            for key in f.keys():
                state_dict[key] = f.get_tensor(key)

        is_lora = all("lora" in k for k in state_dict.keys())
        if not is_lora:
            base_state_dict = state_dict
        else:
            base_state_dict = {}
            # TODO: deal with model_config.base
            with safe_open(model_config.base, framework="pt", device="cpu") as f:
                for key in f.keys():
                    base_state_dict[key] = f.get_tensor(key)

        converted_vae_checkpoint = convert_ldm_vae_checkpoint(base_state_dict, pipeline.vae.config)
        pipeline.vae.load_state_dict(converted_vae_checkpoint)

        converted_unet_checkpoint = convert_ldm_unet_checkpoint(base_state_dict, pipeline.unet.config)
        pipeline.unet.load_state_dict(converted_unet_checkpoint, strict=False)

        #pipeline.text_encoder = convert_ldm_clip_checkpoint(base_state_dict)


        if is_lora:
            pipeline = convert_lora(pipeline, state_dict, alpha=model_config.lora_alpha)

pipeline.to("cuda")


## Aqui se genera la animación

Redacta el prompt, tiene que ser en inglés :)

In [ ]:
# @title

n_frames = 24 # @param {type:"integer"}
width = 448 # @param {type:"integer"}
height = 320 # @param {type:"integer"}

# TODO: maybe pl.seed_everything could improve determinism
seed = 10788741199826055526 # @param {type:"integer"}
steps = 30 # @param {type:"integer"}
guidance_scale = 7.5 # @param {type:"number"}
prompt = "an astronaut, floating at the space with the planet earth background" # @param {type:"string"}
negative_prompt = "stationary, motionless, boring, watermark, trademark, copyright, text, shutterstock" # @param {type:"string"}

outname = "sample.gif" # @param {type:"string"}
outpath = f"./{outname}"

sample = pipeline(
    prompt=prompt,
    negative_prompt     = negative_prompt,
    num_inference_steps = steps,
    guidance_scale      = guidance_scale,
    width               = width,
    height              = height,
    video_length        = n_frames,
).videos

samples = torch.concat([sample])

save_videos_grid(samples, outpath , n_rows=1)

from IPython.display import Image

Image(outpath)

## PROYECTO ANIMACIóN

In [ ]:
%%capture
# @title Instala paquete para convertir de gif a video mp4 y extraer los frames

!pip install moviepy ffmpeg-python

In [ ]:
# @title convierte el gif a video
from moviepy.editor import *


# Convertir GIF a MP4
outname_mp4 = "sample.mp4"
clip = VideoFileClip(outpath)
clip.write_videofile(outname_mp4, codec="libx264", fps=120)

# Para visualizar el video en loop en el cuaderno
from IPython.display import HTML
from base64 import b64encode

mp4 = open(outname_mp4,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=520 controls loop>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# @title Extracción de Frames del video
## @subtitle Basado en el siguiente cuaderno: VideoFrameandAudioextraction [ 'https://colab.research.google.com/drive/1OJIVza67wA4EiFUaJPX2lhEU7y_9iayM#scrollTo=nbVXuLyNwRot&line=2&uniqifier=1']
#################
# Editor de video #
#################
from moviepy.editor import *
import ffmpeg
import os
from pathlib import Path

# Convertir GIF a MP4
outname = "sample.gif"
outpath = f"./{outname}"
outname_mp4 = "sample.mp4"
clip = VideoFileClip(outpath)
clip.write_videofile(outname_mp4, codec="libx264", fps=24)  # Asegúrate de que el FPS sea correcto para tu video

# Función modificada para extraer todos los frames del video MP4
def extract_frames_from_video(video_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    ffmpeg.input(video_path).output(f'{output_dir}/frame%04d.jpg', qscale=0).run()

# Directorio donde se guardarán los frames extraídos
frames_output_dir = "./frames"
# Extraer todos los frames del video MP4
extract_frames_from_video(outname_mp4, frames_output_dir)

# Mostrar información sobre los frames extraídos
print(f"Frames extraídos en {frames_output_dir}")

import os
from IPython.display import Image, display

# Ruta al directorio donde se encuentran los frames
frames_directory = './frames'
frames = [os.path.join(frames_directory, f) for f in os.listdir(frames_directory) if f.endswith('.jpg')]

# Ordena los frames por nombre, si es necesario
frames.sort()

# Visualiza los frames
for frame_path in frames:
    display(Image(filename=frame_path))





